In [1]:
#!/usr/bin/env python

# coding: utf-8


import numpy as np

import matplotlib.pyplot as plt

import xarray as xr

import sklearn

import sklearn.model_selection

import keras

from keras import layers

import keras_tuner

import tensorflow as tf

import tensorflow.keras.backend as K

keras.utils.set_random_seed(812)

2025-01-31 01:33:42.645899: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 01:33:42.665186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738312422.685882   49857 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738312422.692118   49857 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 01:33:42.714669: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
var_list = ["cape", "crr",  "ie", "ishf", "lsrr",  "sp", "sstk", "tcw", "tcwv", "ttr" ] #ERA5 variables

aew_subset = "12hr_before" #time of interest

plevel_list = [False, False,  False, False, False,   False, False, False, False,  False ] #pressure levels of variables

In [3]:
def add_dim(ds):
    """
    Preprocessing help when opening netcdf files.
    """
    return ds.assign_coords({"sample": 1}).expand_dims(dim={"sample": 1}).drop_vars(
        'utc_date').drop_vars("latitude").drop_vars('longitude')

In [4]:
def open_files(list_of_vars, aew_subset="12hr_before",

              directory="/glade/derecho/scratch/rmandava/AEW_time_location_files/",

              plevel_list=None):

   """

   Help opening variables for training.

   """

   datas = {}


   for num, var in enumerate(list_of_vars):


         if not plevel_list:


           datas[var] = xr.open_mfdataset(

           f'{directory}/{var}/aew_{aew_subset}_*.nc',

           preprocess=add_dim,

           concat_dim="sample",

           combine="nested",

           )
         if plevel_list:


              if not plevel_list[num]:


                 datas[var] = xr.open_mfdataset(

                 f'{directory}/{var}/aew_{aew_subset}_*.nc',

                 preprocess=add_dim,

                 concat_dim="sample",

                 combine="nested",

                 )
              if plevel_list[num]:


                 datas[f'{var}_{int(plevel_list[num])}'] = xr.open_mfdataset(

                 f'{directory}/{var}/aew_{aew_subset}_{int(plevel_list[num])}_*.nc',

                 preprocess=add_dim,

                 concat_dim="sample",

                 combine="nested",

                 )
   return datas

In [6]:
for var_name, dataset in datas.items():
    print(f"Dataset: {var_name}")
    print(dataset)

Dataset: cape
<xarray.Dataset> Size: 11MB
Dimensions:  (sample: 2750, latitude: 32, longitude: 32)
Coordinates:
    time     (sample) datetime64[ns] 22kB 1979-06-02T18:00:00 ... 2023-10-27T...
  * sample   (sample) int64 22kB 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
Dimensions without coordinates: latitude, longitude
Data variables:
    CAPE     (sample, latitude, longitude) float32 11MB dask.array<chunksize=(1, 32, 32), meta=np.ndarray>
    label    (sample) float64 22kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB1 data to netCDF4.
    NETCDF_VERSION:       4.6.1
    CONVERSION_PLATFORM:  Linux r2i5n6 3.12.62-60.64.8-default #1 SMP Tue Oct...
    CONVERSION_DATE:      Thu May 16 09:28:20 MDT 2019
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:    

In [ ]:
import pandas as pd

# List of possible temporal coordinates
temporal_coords = ['time', 'forecast_initial_time', 'forecast_hour']

# Open the datasets
datas = open_files(
    list_of_vars=var_list,
    aew_subset=aew_subset,
    directory="/glade/derecho/scratch/rmandava/AEW_time_location_files/",
    plevel_list=plevel_list
)

# Combine the datasets into a single DataFrame
combined_data = pd.DataFrame()

for var_name, dataset in datas.items():
    # Identify the available temporal coordinate
    temporal_var = None
    for coord in temporal_coords:
        if coord in dataset.coords:
            temporal_var = coord
            break
    
    if temporal_var is None:
        raise KeyError(f"No recognized temporal coordinate found in the dataset '{var_name}'.")

    # Rename the temporal coordinate to 'time' for consistency
    dataset = dataset.rename({temporal_var: 'time'})

    # Convert the dataset to a DataFrame
    df = dataset.to_dataframe().reset_index()

    # Merge the DataFrames
    if combined_data.empty:
        combined_data = df
    else:
        combined_data = pd.merge(combined_data, df, on=['sample', 'time'], how='outer')

# Save the combined DataFrame to a CSV file
combined_data.to_csv('combined_data.csv', index=False)

print("Data has been successfully converted to CSV.")
